In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf

# from keras.utils.np_utils import to_categorical
import sklearn
from sklearn.linear_model import LinearRegression

In [73]:
df = pd.read_csv('data/data.csv', sep=';')
# sort data by team and year
df = df.sort_values(['team', 'year'], ascending=[True, True]).reset_index(drop=True)
df.tail()

,year,team,standing
244,2019,Wolves,7
245,2020,Wolves,7
246,2021,Wolves,13
247,2022,Wolves,10
248,2023,Wolves,13


In [40]:
df_dummies = pd.get_dummies(df, columns=['team'])
df_dummies.head()


,year,standing,team_Arsenal,team_Aston Villa,team_Bournemouth,team_Brentford,team_Brighton,team_Burnley,team_Cardiff City,team_Chelsea,...,team_Sheffield United,team_Southampton,team_Stoke City,team_Sunderland,team_Swansea,team_Tottenham,team_Watford,team_West Brom,team_West Ham,team_Wolves
0,2015,3,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2016,2,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2017,5,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2018,6,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2019,5,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
df_filtered = df_dummies.drop(columns=['year'])
df_filtered.head()

In [42]:
vals = df.values
sequences = 2
teams = np.unique(df['team'].values)
print(teams)
X = []
y = []
X_test = []
X_test_teams = []

df_dummies = pd.get_dummies(df, columns=['team'])
df_filtered = df_dummies.drop(columns=['year'])

for team in teams:
    df_team = df_filtered.loc[df['team'] == team]
    #print(df_team)
    team_standings = df_team.values
    # print(f"len(team_standings) {len(team_standings)}")
    for i in range(0, len(team_standings)):
        # print(f"team {team} {i}th year")
        if i + sequences < len(team_standings):
            x_values = team_standings[i:i+sequences]
            # print(x_values)
            # X.append(x_values)
            y_value = team_standings[i+sequences, 0]
            #print(y_value)
            y.append(y_value)
        elif i + sequences == len(team_standings):
            x_values = team_standings[i:i+sequences]
            X_test.append(x_values)
            print(f"team {team}")
            X_test_teams.append(team)

X = np.array(x_values)
y = np.array(y)
X_test = np.array(X_test)
print(X.shape)
print(y.shape)
print(X_test.shape)

['Arsenal' 'Aston Villa' 'Bournemouth' 'Brentford' 'Brighton' 'Burnley'
 'Cardiff City' 'Chelsea' 'Crystal Palace' 'Everton' 'Fulham'
 'Huddersfield' 'Hull City' 'Leeds United' 'Leicester City' 'Liverpool'
 'Man City' 'Man United' 'Middlesbrough' 'Newcastle' 'Norwich City'
 'Nottm Forest' 'Sheffield United' 'Southampton' 'Stoke City' 'Sunderland'
 'Swansea' 'Tottenham' 'Watford' 'West Brom' 'West Ham' 'Wolves']
team Arsenal
team Aston Villa
team Bournemouth
team Brighton
team Burnley
team Cardiff City
team Chelsea
team Crystal Palace
team Everton
team Fulham
team Huddersfield
team Hull City
team Leeds United
team Leicester City
team Liverpool
team Man City
team Man United
team Newcastle
team Norwich City
team Sheffield United
team Southampton
team Stoke City
team Sunderland
team Swansea
team Tottenham
team Watford
team West Brom
team West Ham
team Wolves
(2, 33)
(188,)
(29, 2, 33)


In [43]:
print(y)

[ 5  6  5  8  8  5  2 21 21 21 17 11 14  7  9 12 21 14 18 21 21 15 21 15
 17 15 16  9  6 16  7 15 10 17 18 21 18  1  5  3  4  4  3 12 14 11 12 14
 14 12 11  7  8  8 12 10 16 17 21 19 21 18 21 10 21 16 20 21 21 21 18 21
 21 21 21 21 21 21 21 21  9 17 19 12  9 21  9  5  5  8 18  4  4  2  1  3
  2  5  3  1  1  2  1  1  1  6  2  6  3  2  6  3 10 13 13 12 11  4 21 21
 21 20 21 20 21 21 21 21  9 20 21 21  8 17 16 11 15 15 20 13 19 21 21 21
 21 21 20 21 21 21 21 21 15 18  2  3  4  6  7  4  7 14 11 19 21 19 21 10
 20 21 21 19 32 21 11 13 10 16  6  7 14 21 21  7  7 13 10 13]


In [44]:
print(X_test)

[[[5 True False ... False False False]
  [2 True False ... False False False]]

 [[14 False True ... False False False]
  [7 False True ... False False False]]

 [[21 False False ... False False False]
  [15 False False ... False False False]]

 ...

 [[32 False False ... True False False]
  [21 False False ... True False False]]

 [[7 False False ... False True False]
  [14 False False ... False True False]]

 [[10 False False ... False False True]
  [13 False False ... False False True]]]


In [11]:
def linear_transform(X):
    if len(X.shape) == 3:
        if X.shape[1] == 1:
            X_linear = np.squeeze(X, 1)
        else:
            X_linear = X.reshape((X.shape[0], X.shape[1]* X.shape[2]))
    return X_linear

In [12]:
X_linear = linear_transform(X)
print(X_linear.shape)

(188, 66)


In [13]:
print(X_linear.shape)
print(y.shape)
reg = LinearRegression().fit(X_linear, y)
reg.score(X_linear, y)

(188, 66)
(188,)


0.7605010407888186

In [14]:
X_test_linear = linear_transform(X_test)
print(X_test_linear.shape)

(29, 66)


In [15]:
print(X_test_teams)

['Arsenal', 'Aston Villa', 'Bournemouth', 'Brighton', 'Burnley', 'Cardiff City', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Huddersfield', 'Hull City', 'Leeds United', 'Leicester City', 'Liverpool', 'Man City', 'Man United', 'Newcastle', 'Norwich City', 'Sheffield United', 'Southampton', 'Stoke City', 'Sunderland', 'Swansea', 'Tottenham', 'Watford', 'West Brom', 'West Ham', 'Wolves']


In [16]:
result = reg.predict(X_test_linear)
print(np.array(result))

[ 4.75740051 13.30683899 16.3177948  11.28334045 16.41938782 17.32469177
  6.23901367 12.03869629 13.04429626 16.10661316 20.2842865  20.67167664
 18.34854126 12.852005    3.23130798  1.17868042  3.83911133  8.37016296
 20.80323792 19.67070007 16.45393372 20.57391357 21.12001038 17.81568909
  5.27507019 18.62628174 21.82701111 11.6658783  12.49143982]


In [17]:
inds = np.argsort(result, axis=0)
print(inds)

[15 14 16  0 24  6 17  3 27  7 28 13  8  1  9  2  4 20  5 23 12 25 19 10
 21 11 18 22 26]


In [18]:
for i, ind in enumerate(inds):
    team = X_test_teams[ind]
    res = result[ind]
    print(f"{i+1}th team {team} is {round(res, 2)}")

1th team Man City is 1.18
2th team Liverpool is 3.23
3th team Man United is 3.84
4th team Arsenal is 4.76
5th team Tottenham is 5.28
6th team Chelsea is 6.24
7th team Newcastle is 8.37
8th team Brighton is 11.28
9th team West Ham is 11.67
10th team Crystal Palace is 12.04
11th team Wolves is 12.49
12th team Leicester City is 12.85
13th team Everton is 13.04
14th team Aston Villa is 13.31
15th team Fulham is 16.11
16th team Bournemouth is 16.32
17th team Burnley is 16.42
18th team Southampton is 16.45
19th team Cardiff City is 17.32
20th team Swansea is 17.82
21th team Leeds United is 18.35
22th team Watford is 18.63
23th team Sheffield United is 19.67
24th team Huddersfield is 20.28
25th team Stoke City is 20.57
26th team Hull City is 20.67
27th team Norwich City is 20.8
28th team Sunderland is 21.12
29th team West Brom is 21.83


# LSTM

In [83]:
def set_dataset(X, y):
    train_ds = tf.data.Dataset.from_tensor_slices((X, y))
    return train_ds

def preprocessing(df, sequences):
    vals = df.values
    teams = np.unique(df['team'].values)
    print(teams)
    X = []
    y = []
    X_test = []
    X_test_teams = []

    # df_dummies = pd.get_dummies(df, columns=['team'])
    df_filtered = df.drop(columns=['year'])

    for team in teams:
        df_team = df_filtered.loc[df['team'] == team]
        # print(df_team)
        team_standings = df_team.standing.values
        print(f"team_standings {team_standings}")
        for i in range(0, len(team_standings)):
            print(f"team {team} {i}th year")
            if i + sequences < len(team_standings):
                x_values = team_standings[i:i+sequences]
                print(f"x_values {x_values}")
                X.append(x_values)
                y_value = team_standings[i+sequences]
                print(f"y_value {y_value}")
                y.append(y_value)
            elif i + sequences == len(team_standings):
                x_values = team_standings[i:i+sequences]
                X_test.append(x_values)
                print(f"team {team}")
                X_test_teams.append(team)

    X = np.array(X)
    X = X.reshape(X.shape[0], X.shape[1], 1)
    y = np.array(y)
    X_test = np.array(X_test)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
    print(X.shape)
    print(y.shape)
    print(X_test.shape)
    return X, y, X_test, X_test_teams

In [84]:
X, y, X_test, X_test_teams = preprocessing(df, n_timesteps)

['Arsenal' 'Aston Villa' 'Bournemouth' 'Brentford' 'Brighton' 'Burnley'
 'Cardiff City' 'Chelsea' 'Crystal Palace' 'Everton' 'Fulham'
 'Huddersfield' 'Hull City' 'Leeds United' 'Leicester City' 'Liverpool'
 'Man City' 'Man United' 'Middlesbrough' 'Newcastle' 'Norwich City'
 'Nottm Forest' 'Sheffield United' 'Southampton' 'Stoke City' 'Sunderland'
 'Swansea' 'Tottenham' 'Watford' 'West Brom' 'West Ham' 'Wolves']
team_standings [3 2 5 6 5 8 8 5 2]
team Arsenal 0th year
x_values [3 2 5]
y_value 6
team Arsenal 1th year
x_values [2 5 6]
y_value 5
team Arsenal 2th year
x_values [5 6 5]
y_value 8
team Arsenal 3th year
x_values [6 5 8]
y_value 8
team Arsenal 4th year
x_values [5 8 8]
y_value 5
team Arsenal 5th year
x_values [8 8 5]
y_value 2
team Arsenal 6th year
team Arsenal
team Arsenal 7th year
team Arsenal 8th year
team_standings [17 20 21 21 21 17 11 14  7]
team Aston Villa 0th year
x_values [17 20 21]
y_value 21
team Aston Villa 1th year
x_values [20 21 21]
y_value 21
team Aston Villa 2t

In [85]:
def set_lstm(n_timesteps, n_features, hidden, n_outputs):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(hidden, input_shape=(n_timesteps, n_features)))
    model.add(tf.keras.layers.Dropout(0.1))
    #model.add(Dense(hidden, activation='relu'))
    model.add(tf.keras.layers.Dense(n_outputs, activation='relu'))
    # model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['mse'])
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    model.summary()
    return model

def predict(model, X_test):
    lstm_preds = model.predict(X_test)
    inds = np.argsort(lstm_preds[:,0])
    print(inds)
    print(lstm_preds.shape)

    for i, ind in enumerate(inds):
        team = X_test_teams[ind]
        pred = lstm_preds[ind, 0]
        print(f"{i+1}th team {team} is {str(np.round(pred, 1))}")

def train(n_timesteps, n_features, hidden, output_size, batch_size, epochs):
    model = set_lstm(n_timesteps, n_features, hidden, output_size)
    # Compile the model
    # optimizer = tf.keras.optimizers.Adam()
    # model.compile(optimizer=optimizer, loss='mse')

    # train_ds = set_dataset(X, y)
    print(X.shape)
    X_train, X_val = X[:100,:,:], X[100:,:]
    y_train, y_val = y[:100], y[100:]

    model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=2, validation_split=0.2)
    # evaluate model
    _, accuracy = model.evaluate(X_val, y_val, batch_size=batch_size, verbose=0)

    print(accuracy)
    return model

In [87]:
n_timesteps = 3
n_features = 1
hidden = 64
output_size = 1  # labels are from 1-21
batch_size = 4
epochs = 100

X, y, X_test, X_test_teams = preprocessing(df, n_timesteps)

model = train(n_timesteps, n_features, hidden, output_size, batch_size, epochs)


['Arsenal' 'Aston Villa' 'Bournemouth' 'Brentford' 'Brighton' 'Burnley'
 'Cardiff City' 'Chelsea' 'Crystal Palace' 'Everton' 'Fulham'
 'Huddersfield' 'Hull City' 'Leeds United' 'Leicester City' 'Liverpool'
 'Man City' 'Man United' 'Middlesbrough' 'Newcastle' 'Norwich City'
 'Nottm Forest' 'Sheffield United' 'Southampton' 'Stoke City' 'Sunderland'
 'Swansea' 'Tottenham' 'Watford' 'West Brom' 'West Ham' 'Wolves']
team_standings [3 2 5 6 5 8 8 5 2]
team Arsenal 0th year
x_values [3 2 5]
y_value 6
team Arsenal 1th year
x_values [2 5 6]
y_value 5
team Arsenal 2th year
x_values [5 6 5]
y_value 8
team Arsenal 3th year
x_values [6 5 8]
y_value 8
team Arsenal 4th year
x_values [5 8 8]
y_value 5
team Arsenal 5th year
x_values [8 8 5]
y_value 2
team Arsenal 6th year
team Arsenal
team Arsenal 7th year
team Arsenal 8th year
team_standings [17 20 21 21 21 17 11 14  7]
team Aston Villa 0th year
x_values [17 20 21]
y_value 21
team Aston Villa 1th year
x_values [20 21 21]
y_value 21
team Aston Villa 2t

2023-08-09 21:52:27.884627: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8902
2023-08-09 21:52:28.133645: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


32/32 - 4s - loss: 177.1159 - mse: 177.1159 - val_loss: 153.1549 - val_mse: 153.1549 - 4s/epoch - 123ms/step
Epoch 2/100
32/32 - 0s - loss: 102.2041 - mse: 102.2041 - val_loss: 85.7483 - val_mse: 85.7483 - 146ms/epoch - 5ms/step
Epoch 3/100
32/32 - 0s - loss: 59.4819 - mse: 59.4819 - val_loss: 56.9433 - val_mse: 56.9433 - 153ms/epoch - 5ms/step
Epoch 4/100
32/32 - 0s - loss: 38.7682 - mse: 38.7682 - val_loss: 41.4926 - val_mse: 41.4926 - 151ms/epoch - 5ms/step
Epoch 5/100
32/32 - 0s - loss: 28.9900 - mse: 28.9900 - val_loss: 35.5447 - val_mse: 35.5447 - 134ms/epoch - 4ms/step
Epoch 6/100
32/32 - 0s - loss: 25.1354 - mse: 25.1354 - val_loss: 31.4797 - val_mse: 31.4797 - 123ms/epoch - 4ms/step
Epoch 7/100
32/32 - 0s - loss: 20.8156 - mse: 20.8156 - val_loss: 30.3314 - val_mse: 30.3314 - 118ms/epoch - 4ms/step
Epoch 8/100
32/32 - 0s - loss: 18.7433 - mse: 18.7433 - val_loss: 28.7365 - val_mse: 28.7365 - 119ms/epoch - 4ms/step
Epoch 9/100
32/32 - 0s - loss: 18.3209 - mse: 18.3209 - val_los

In [88]:
predict(model, X_test)

[15 16  0 14 17  3 24  1  6  7 28  9 27  2  8 13 23  5 12 20  4 25 18 19
 21 10 22 11 26]
(29, 1)
1th team Man City is 0.7
2th team Man United is 3.1
3th team Arsenal is 4.0
4th team Liverpool is 4.4
5th team Newcastle is 5.1
6th team Brighton is 5.6
7th team Tottenham is 7.2
8th team Aston Villa is 8.9
9th team Chelsea is 8.9
10th team Crystal Palace is 9.5
11th team Wolves is 13.0
12th team Fulham is 14.2
13th team West Ham is 14.7
14th team Bournemouth is 16.1
15th team Everton is 16.2
16th team Leicester City is 16.5
17th team Swansea is 16.8
18th team Cardiff City is 17.7
19th team Leeds United is 17.8
20th team Southampton is 18.0
21th team Burnley is 18.8
22th team Watford is 19.0
23th team Norwich City is 19.1
24th team Sheffield United is 19.1
25th team Stoke City is 19.2
26th team Huddersfield is 19.2
27th team Sunderland is 19.2
28th team Hull City is 19.2
29th team West Brom is 19.5
